In [2]:
import numpy as np
import cupy as cp
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from scipy import interp
from itertools import cycle
from tensorflow.keras import layers, models
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc


#  Data Processing

In [3]:
gear_defects = [0, 35, 76, 77]

In [4]:
# Image size
size = [300, 400]

In [5]:
# Load pre-processed dataset
X_train = np.load('gears_train_300x400_0,76,77,35.npy')
X_test = np.load('gears_test_300x400_0,76,77,35.npy')
y_train = np.load('gears_ytrain_300x400_0,76,77,35.npy')
y_test = np.load('gears_ytest_300x400_0,76,77,35.npy')

# Training 

In [6]:
# Define early stopping and best model checkpoint
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, min_delta=0.1, patience=40)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_weights_only=True, mode='max', verbose=1, save_best_only=True)

In [7]:
# Half filter case
dropout = 0.5
model = models.Sequential()
model.add(layers.Conv2D(32, (11, 11), strides=(4,4), activation='relu', input_shape=(size[0],size[1],1)))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Conv2D(64, (5, 5), strides=(1,1), padding="same", activation='tanh'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Conv2D(128, (3, 3), strides=(1,1), padding="same", activation='tanh'))
model.add(layers.Conv2D(128, (3, 3), strides=(1,1), padding="same", activation='tanh'))
model.add(layers.Conv2D(64, (3, 3), strides=(1,1), padding="same", activation='tanh'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='sigmoid'))
model.add(layers.Dropout(dropout))
model.add(layers.Dense(128, activation='sigmoid'))
model.add(layers.Dense(len(np.unique(y_train))))
model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 73, 98, 32)        3904      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 36, 48, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 36, 48, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 36, 48, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 17, 23, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 17, 23, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 23, 128)       7

In [8]:
# Fitting model with early stopping and checkpoint save
history = model.fit(X_train, y_train, epochs=100, 
                    validation_data=(X_test, y_test), callbacks=[es, mc])

Epoch 1/100
262/262 [==============================] - ETA: 0s - loss: 0.7450 - accuracy: 0.7865
Epoch 00001: val_accuracy improved from -inf to 0.75048, saving model to best_model.h5
262/262 [==============================] - 4s 16ms/step - loss: 0.7450 - accuracy: 0.7865 - val_loss: 0.8837 - val_accuracy: 0.7505
Epoch 2/100
260/262 [============================>.] - ETA: 0s - loss: 0.7404 - accuracy: 0.7880
Epoch 00002: val_accuracy did not improve from 0.75048
262/262 [==============================] - 4s 14ms/step - loss: 0.7398 - accuracy: 0.7883 - val_loss: 0.8850 - val_accuracy: 0.7505
Epoch 3/100
259/262 [============================>.] - ETA: 0s - loss: 0.7371 - accuracy: 0.7873
Epoch 00003: val_accuracy did not improve from 0.75048
262/262 [==============================] - 4s 14ms/step - loss: 0.7349 - accuracy: 0.7883 - val_loss: 0.8934 - val_accuracy: 0.7505
Epoch 4/100
261/262 [============================>.] - ETA: 0s - loss: 0.7386 - accuracy: 0.7882
Epoch 00004: val_ac

Epoch 30/100
260/262 [============================>.] - ETA: 0s - loss: 0.7284 - accuracy: 0.7892
Epoch 00030: val_accuracy did not improve from 0.75048
262/262 [==============================] - 4s 14ms/step - loss: 0.7305 - accuracy: 0.7883 - val_loss: 0.8525 - val_accuracy: 0.7505
Epoch 31/100
261/262 [============================>.] - ETA: 0s - loss: 0.7308 - accuracy: 0.7883
Epoch 00031: val_accuracy did not improve from 0.75048
262/262 [==============================] - 4s 14ms/step - loss: 0.7308 - accuracy: 0.7883 - val_loss: 0.8851 - val_accuracy: 0.7505
Epoch 32/100
260/262 [============================>.] - ETA: 0s - loss: 0.7336 - accuracy: 0.7874
Epoch 00032: val_accuracy did not improve from 0.75048
262/262 [==============================] - 4s 14ms/step - loss: 0.7318 - accuracy: 0.7883 - val_loss: 0.8646 - val_accuracy: 0.7505
Epoch 33/100
260/262 [============================>.] - ETA: 0s - loss: 0.7306 - accuracy: 0.7883
Epoch 00033: val_accuracy did not improve from 

In [12]:
model.save('alexnet_half.h5')

In [13]:
# convert the history.history dict to a pandas DataFrame   
hist_df = pd.DataFrame(history.history) 

# save to json
hist_json_file = 'alexnt_half_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [14]:
# Double filter case
dropout = 0.5
model = models.Sequential()
model.add(layers.Conv2D(128, (11, 11), strides=(4,4), activation='relu', input_shape=(size[0],size[1],1)))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Conv2D(256, (5, 5), strides=(1,1), padding="same", activation='sigmoid'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Conv2D(512, (3, 3), strides=(1,1), padding="same", activation='sigmoid'))
model.add(layers.Conv2D(512, (3, 3), strides=(1,1), padding="same", activation='sigmoid'))
model.add(layers.Conv2D(256, (3, 3), strides=(1,1), padding="same", activation='sigmoid'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='sigmoid'))
model.add(layers.Dropout(dropout))
model.add(layers.Dense(512, activation='sigmoid'))
model.add(layers.Dense(len(np.unique(y_train))))
model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 73, 98, 128)       15616     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 48, 128)       0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 36, 48, 128)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 36, 48, 256)       819456    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 17, 23, 256)       0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 17, 23, 256)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 17, 23, 512)      

In [15]:
# Fitting model with early stopping and checkpoint save
history = model.fit(X_train, y_train, epochs=100, 
                    validation_data=(X_test, y_test), callbacks=[es, mc])

Epoch 1/100
261/262 [============================>.] - ETA: 0s - loss: 0.7698 - accuracy: 0.7858
Epoch 00001: val_accuracy did not improve from 0.75048
262/262 [==============================] - 13s 48ms/step - loss: 0.7702 - accuracy: 0.7857 - val_loss: 0.9006 - val_accuracy: 0.7505
Epoch 2/100
261/262 [============================>.] - ETA: 0s - loss: 0.7505 - accuracy: 0.7878
Epoch 00002: val_accuracy did not improve from 0.75048
262/262 [==============================] - 12s 47ms/step - loss: 0.7494 - accuracy: 0.7883 - val_loss: 0.8826 - val_accuracy: 0.7505
Epoch 3/100
261/262 [============================>.] - ETA: 0s - loss: 0.7429 - accuracy: 0.7886
Epoch 00003: val_accuracy did not improve from 0.75048
262/262 [==============================] - 12s 47ms/step - loss: 0.7437 - accuracy: 0.7883 - val_loss: 0.8457 - val_accuracy: 0.7505
Epoch 4/100
261/262 [============================>.] - ETA: 0s - loss: 0.7375 - accuracy: 0.7882
Epoch 00004: val_accuracy did not improve from 0

Epoch 30/100
261/262 [============================>.] - ETA: 0s - loss: 0.7365 - accuracy: 0.7882
Epoch 00030: val_accuracy did not improve from 0.75048
262/262 [==============================] - 12s 47ms/step - loss: 0.7365 - accuracy: 0.7883 - val_loss: 0.8662 - val_accuracy: 0.7505
Epoch 31/100
261/262 [============================>.] - ETA: 0s - loss: 0.7336 - accuracy: 0.7882
Epoch 00031: val_accuracy did not improve from 0.75048
262/262 [==============================] - 12s 47ms/step - loss: 0.7335 - accuracy: 0.7883 - val_loss: 0.8767 - val_accuracy: 0.7505
Epoch 32/100
261/262 [============================>.] - ETA: 0s - loss: 0.7366 - accuracy: 0.7883
Epoch 00032: val_accuracy did not improve from 0.75048
262/262 [==============================] - 12s 47ms/step - loss: 0.7369 - accuracy: 0.7883 - val_loss: 0.8509 - val_accuracy: 0.7505
Epoch 33/100
261/262 [============================>.] - ETA: 0s - loss: 0.7360 - accuracy: 0.7886
Epoch 00033: val_accuracy did not improve fr

In [17]:
model.save('alexnet_double.h5')

In [18]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = 'alexnt_double_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)